In [2]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset, Dataset

import pandas as pd
from sklearn.model_selection import train_test_split
import dill

In [8]:

en = spacy.load('en')
de = spacy.load('de')

In [12]:
def data_load():

    data_en = open('data/en-de/train.en', encoding='utf-8').read().split('\n')
    data_de = open('data/en-de/train.de', encoding='utf-8').read().split('\n')

    raw_data = {'EN' : [line for line in data_en], 'DE': [line for line in data_de]} 

    return raw_data


In [3]:
def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]
def tokenize_de(sentence):
    return [tok.text for tok in de.tokenizer(sentence)]


In [10]:
def data_tokenize_save():

    print('raw data loading...')

    raw_data = data_load()

    df = pd.DataFrame(raw_data, columns=["EN", "DE"])

    df['en_len'] = df['EN'].str.count(' ')
    df['de_len'] = df['DE'].str.count(' ')
    df = df.query('de_len < 80 & en_len < 80') 
    df = df.query('de_len < en_len * 1.5 & de_len * 1.5 > en_len')

    train, val = train_test_split(df, test_size=0.2)
    train.to_csv("data/en-de/train.csv", index=False)
    val.to_csv("data/en-de/val.csv", index=False)

    print('tokenizing...') 

    data_fields = [('EN', EN_TEXT), ('DE', DE_TEXT)]
    train,val = TabularDataset.splits(path='data/en-de', train='train.csv', validation='val.csv', format='csv', fields=data_fields)

    with open('data/en-de/train.dill', 'wb') as f:
        dill.dump(train.examples, f)
    with open('data/en-de/val.dill', 'wb') as f:
        dill.dump(val.examples, f)


In [6]:
EN_TEXT = Field(tokenize=tokenize_en, init_token = "<sos>", eos_token = "<eos>")
DE_TEXT = Field(tokenize=tokenize_de, init_token = "<sos>", eos_token = "<eos>")

In [14]:
print('data loading...')

data_fields = [('EN', EN_TEXT), ('DE', DE_TEXT)]
train,val = TabularDataset.splits(path='data/en-de', train='train_dummy.csv', validation='val_dummy.csv', format='csv', fields=data_fields)

with open('data/en-de/val.dill', 'rb') as f:
    val.examples = dill.load(f) 


data loading...


In [15]:
EN_TEXT.build_vocab(val)
DE_TEXT.build_vocab(val)

In [17]:
val_iter = BucketIterator(val, batch_size=32,sort_key=lambda x: len(x.EN), shuffle=True)

In [51]:
batch = next(iter(val_iter))
input_seq = batch.EN.transpose(0,1)
input_pad = EN_TEXT.vocab.stoi['<pad>']

In [52]:
input_seq

tensor([[   2, 3653,   36,  ...,    1,    1,    1],
        [   2,  326,   25,  ...,    1,    1,    1],
        [   2,  183,   24,  ...,    1,    1,    1],
        ...,
        [   2,   22, 1810,  ...,    1,    1,    1],
        [   2,   23, 2147,  ...,    1,    1,    1],
        [   2,   62,  579,  ...,    1,    1,    1]])

In [56]:
input_msk = (input_seq != input_pad)

In [57]:
input_msk

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.uint8)

In [58]:
input_seq.masked_fill_(input_msk,-1e9)

tensor([[-1000000000, -1000000000, -1000000000,  ...,           1,
                   1,           1],
        [-1000000000, -1000000000, -1000000000,  ...,           1,
                   1,           1],
        [-1000000000, -1000000000, -1000000000,  ...,           1,
                   1,           1],
        ...,
        [-1000000000, -1000000000, -1000000000,  ...,           1,
                   1,           1],
        [-1000000000, -1000000000, -1000000000,  ...,           1,
                   1,           1],
        [-1000000000, -1000000000, -1000000000,  ...,           1,
                   1,           1]])

In [37]:
input_seq.size()

torch.Size([32, 58])

In [40]:
input_msk = (input_seq != input_pad)

In [42]:
input_msk.size()

torch.Size([32, 58])

In [4]:
EN_TEXT = Field(tokenize=tokenize_en, init_token = "<sos>", eos_token = "<eos>")
DE_TEXT = Field(tokenize=tokenize_de, init_token = "<sos>", eos_token = "<eos>")

In [ ]:
EN = pd.read_csv('data/en-de/train.en',sep='\n',names = ['text'])
DE = pd.read_csv('data/en-de/train.de',sep='\n',names = ['text'])

In [ ]:
EN_test = EN[:-893768]
EN_val = EN[-893768:]
DE_test = DE[:-893768]
DE_val = DE[-893768:]

In [ ]:
EN_test.to_csv("data/en-de/train_csv.en",mode="w",header=False)
DE_test.to_csv("data/en-de/train_csv.de",mode="w",header=False)
EN_val.to_csv("data/en-de/val_csv.en",mode="w",header=False)
DE_val.to_csv("data/en-de/val_csv.de",mode="w",header=False)

In [13]:
from torchtext.data import Field

In [14]:
import spacy

In [15]:
from torchtext.data import TabularDataset

In [16]:
from torchtext.datasets import Multi30k

In [17]:
# for tokenizing the english sentences
spacy_en = spacy.load('en')
# for tokenizing the german sentences
spacy_de = spacy.load('de')

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [18]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]    


def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [19]:
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True, batch_first=True)

In [81]:
train, val= TabularDataset.splits(
        path='data/en-de', train='train_csv.en',
        validation='val_csv.en', format='csv',fields=SRC)

TypeError: zip argument #1 must support iteration

In [20]:
import pandas as pd

In [59]:
EN = pd.read_csv('data/en-de/train.en',sep='\n',names = ['text'])
DE = pd.read_csv('data/en-de/train.de',sep='\n',names = ['text'])

In [60]:
EN_test = EN[:-893768]
EN_val = EN[-893768:]
DE_test = DE[:-893768]
DE_val = DE[-893768:]

In [69]:
EN_test.to_csv("data/en-de/train_csv.en",mode="w",header=False)
DE_test.to_csv("data/en-de/train_csv.de",mode="w",header=False)
EN_val.to_csv("data/en-de/val_csv.en",mode="w",header=False)
DE_val.to_csv("data/en-de/val_csv.de",mode="w",header=False)

In [49]:
print(X['text'][])
print(Y['text'][4468838])

Three in ten South Africans are younger than 15 , meaning that they did not live a day under apartheid .
Drei von zehn Südafrikanern sind jünger als 15 und das bedeutet , dass sie nicht einen Tag unter der Apartheid gelebt haben .


In [56]:
X_VAL = X[:-893768]

In [58]:
X_VAL.head()

,text
0,iron cement is a ready for use paste which is ...
1,iron cement protects the ingot against the hot...
2,"a fire restant repair cement for fire places ,..."
3,Construction and repair of highways and ...
4,An announcement must be commercial character .


In [52]:
A = [1,2,3,4,5,6,7,8,9,10]

In [55]:
A[:-2]

[1, 2, 3, 4, 5, 6, 7, 8]

In [54]:
A[-2:]

[9, 10]

In [51]:
4468840*0.2

893768.0

In [45]:
X['text'][4468839]

'Somehow Zuma must find a way to honor his own generation ’ s commitment to racial justice and national liberation , while empowering the masses who daily suffer the sting of class differences and yearn for material gain .'

In [46]:
Y['text'][4468839]

'Irgendwie muss Zuma einen Weg finden , einerseits das Engagement seiner Generation hinsichtlich ethnischer Gerechtigkeit und nationaler Befreiung zu würdigen und andererseits den Massen , die täglich unter Klassenunterschieden leiden und sich nach materiellen Verbesserungen sehnen , mehr Mitwirkungsmöglichkeiten einzuräumen .'

In [15]:
train_data = Multi30k.splits(path='./data/en-de',
                                   exts = ('.en','.de'),
                                   fields=(SRC,TRG))

FileNotFoundError: [Errno 2] No such file or directory: './data/en-de/val.en'

In [1]:
type(train_data)

NameError: name 'train_data' is not defined

In [32]:
from torchtext.data import Field, BucketIterator, TabularDataset, Dataset
import spacy
import torchtext
import dill

In [25]:
!python -m spacy download en

     |████████████████████████████████| 12.0MB 615kB/s eta 0:00:01
  Stored in directory: /tmp/pip-ephem-wheel-cache-q6ezcdt4/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/gwanghoon/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/home/gwanghoon/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [28]:
!python -m spacy download de

     |████████████████████████████████| 14.9MB 751kB/s eta 0:00:01
  Stored in directory: /tmp/pip-ephem-wheel-cache-y6s57l73/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/home/gwanghoon/anaconda3/lib/python3.7/site-packages/de_core_news_sm -->
/home/gwanghoon/anaconda3/lib/python3.7/site-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [29]:
en = spacy.load('en')
de = spacy.load('de')

In [30]:
def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]
def tokenize_de(sentence):
    return [tok.text for tok in de.tokenizer(sentence)]
EN_TEXT = Field(tokenize=tokenize_en, init_token = "<sos>", eos_token = "<eos>")
DE_TEXT = Field(tokenize=tokenize_de, init_token = "<sos>", eos_token = "<eos>")

In [33]:
data_fields = [('EN', EN_TEXT), ('DE', DE_TEXT)]
train,val = TabularDataset.splits(path='data/en-de', train='train_dummy.csv', validation='val_dummy.csv', format='csv', fields=data_fields)

with open('data/en-de/train.dill', 'rb') as f:
    train.examples = dill.load(f) 
with open('data/en-de/val.dill', 'rb') as f:
    val.examples = dill.load(f) 


In [34]:
EN_TEXT.build_vocab(train, val)
DE_TEXT.build_vocab(train, val)

In [39]:
EN_TEXT.vocab

In [13]:
len(EN_TEXT.vocab)

657174

In [14]:
len(DE_TEXT.vocab)

1399835

In [20]:
import math

In [21]:
math.sqrt(9)

3.0

In [34]:
import torch
import torch.nn as nn
import numpy as np

In [24]:
loss = nn.MSELoss(reduction ='sum')
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()